### Wangling Data
In order to answer the questions asked in step 1, we are going to define some functions to start working with the data to get what we want.

At this point, it would be interesting to use a database to facilitate the integration of the two datasets, however, I will be using only python to browse the files for learning purposes.

In [14]:
import re
import os
import pandas as pd
from utils import print_json
from statisticalAnalysis import meanTest, avgGrade

In [3]:

d = []
for j in range(5):
    nome = 'col'+str(j)
    vet = []
    for i in range(100):
        vet.append(i*(j+1))
    d.append([nome, vet])
d = dict(d)
df = pd.DataFrame(data=d)
print(df.shape)


df = df.drop(df.index.difference([x for x in range(0, len(df), 10)]))
print(df.shape)
df

(100, 5)
(10, 5)


,col0,col1,col2,col3,col4
0,0,0,0,0,0
10,10,20,30,40,50
20,20,40,60,80,100
30,30,60,90,120,150
40,40,80,120,160,200
50,50,100,150,200,250
60,60,120,180,240,300
70,70,140,210,280,350
80,80,160,240,320,400
90,90,180,270,360,450


In [4]:
# Processed dataset path ENEM
enem_analysis_path = os.getcwd() + '/data/analysis/enem_analysis.csv'
enem_analysis_df = pd.read_csv(enem_analysis_path, delimiter=";")
enem_analysis_df = enem_analysis_df.drop(enem_analysis_df.index.difference([x for x in range(0, len(enem_analysis_df), 100)]))

print(enem_analysis_df.shape)
enem_analysis_df

(55138, 41)


,registration,city_residence_code,city_residence,state_residence_code,state_residence,age,gender,matiral_status,color_race,nationality,...,presence_natural_science,presence_human_science,presence_languages,presence_math,grade_natural_science,grade_human_science,grade_languages,grade_math,essay_status,grade_essay
0,180008202043,5300108,Brasília,53,DF,44.0,M,1.0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,180008787997,2111300,São Luís,21,MA,17.0,M,0.0,3,1,...,1.0,1.0,1.0,1.0,492.2,659.1,587.2,590.5,1.0,920.0
200,180008202064,5107958,Tangará da Serra,51,MT,28.0,F,0.0,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,180008411132,4305108,Caxias do Sul,43,RS,44.0,F,0.0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,180008196190,1100338,Nova Mamoré,11,RO,27.0,F,0.0,1,1,...,1.0,1.0,1.0,1.0,543.0,560.5,572.1,651.3,1.0,480.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513300,180013592356,1302504,Manacapuru,13,AM,19.0,M,0.0,1,1,...,0.0,1.0,1.0,0.0,0.0,403.0,447.6,0.0,1.0,380.0
5513400,180013075385,1100056,Cerejeiras,11,RO,18.0,M,0.0,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5513500,180013831993,5002704,Campo Grande,50,MS,32.0,M,0.0,3,1,...,1.0,1.0,1.0,1.0,438.9,514.4,422.3,492.9,1.0,260.0
5513600,180013663323,2408003,Mossoró,24,RN,50.0,M,0.0,3,1,...,0.0,1.0,1.0,0.0,0.0,484.4,489.9,0.0,4.0,0.0


In [5]:
def meanTest(test):
    import re
    grades = []
    for i, r in test.iterrows():
        if r[list(filter(lambda col: re.search("(presence|status)", col), test.columns))[0]] == 1.0:
            grades.append(r[list(filter(lambda col: re.search("grade", col), test.columns))[0]])
    return { 'mean': sum(grades)/len(grades)
           ,'min': min(grades)
           ,'max': max(grades)
         }
        

def avgGrade(enem, onlyPresent=True):
    """
        Returns average grade of each test
        Attributes:
            enem (pandas dataframe): ENEM dataframe that contains all grades
            onlyPresent (boolean): TRUE if you only want the grades of the students who took the test. FALSE if you want all the grades
    """
    if onlyPresent:
        return { 'natural_science': meanTest(test = enem[['presence_natural_science'
                                                   , 'grade_human_science'
                                                  ]])
                ,'human_science': meanTest(test = enem[['presence_languages'
                                                 , 'grade_natural_science'
                                                ]])
                ,'languages': meanTest(test = enem[['presence_human_science'
                                             , 'grade_languages'
                                            ]])
                ,'math': meanTest(test = enem[['presence_math'
                                        , 'grade_math'
                                       ]])
                ,'essay': meanTest(test = enem[['essay_status'
                                         , 'grade_essay'
                                        ]])
               } 
    else:
        return { 'natural_science': { 'mean': enem['grade_natural_science'].mean()
                                     ,'min': enem['grade_natural_science'].min()
                                     ,'max': enem['grade_natural_science'].max()
                                    }
                ,'human_science': { 'mean': enem['grade_human_science'].mean()
                                   ,'min': enem['grade_human_science'].min()
                                   ,'max': enem['grade_human_science'].max()
                                  }
                ,'languages': { 'mean': enem['grade_languages'].mean()
                               ,'min': enem['grade_languages'].min()
                               ,'max': enem['grade_languages'].max()
                              }
                ,'math': { 'mean': enem['grade_math'].mean()
                          ,'min': enem['grade_math'].min()
                          ,'max': enem['grade_math'].max()
                         }
                ,'essay': { 'mean': enem['grade_essay'].mean()
                           ,'min': enem['grade_essay'].min()
                           ,'max': enem['grade_essay'].max()
                          }
               }

In [16]:
# from utils import print_json

res = avgGrade(enem_analysis_df, onlyPresent=False)
# print(json.dumps(res, indent=2))

print_json(res)

res = avgGrade(enem_analysis_df)
print_json(res)

{"natural_science": {"mean": 349.6525753563786, "min": 0.0, "max": 819.0}, "human_science": {"mean": 427.94519387718094, "min": 0.0, "max": 821.0}, "languages": {"mean": 396.4180456309623, "min": 0.0, "max": 792.7}, "math": {"mean": 379.209840763176, "min": 0.0, "max": 988.4}, "essay": {"mean": 382.5169574522108, "min": 0.0, "max": 980.0}}
{"natural_science": {"mean": 568.4935762469341, "min": 0.0, "max": 821.0}, "human_science": {"mean": 462.5714869870632, "min": 0.0, "max": 819.0}, "languages": {"mean": 525.7540337711079, "min": 0.0, "max": 792.7}, "math": {"mean": 534.2618612019631, "min": 0.0, "max": 988.4}, "essay": {"mean": 521.3372552896974, "min": 60.0, "max": 980.0}}
